In [1]:
import pandas as pd
import numpy as np

In [4]:
loop_pred_100 = pd.read_csv('./100loop_result/test_pred.csv')
loop_prob_100 = pd.read_csv('./100loop_result/test_prob.csv')
meta_data = np.load('./100loop_result/val.npz', allow_pickle=True)
tmpl = pd.read_csv('../testing/submission_template_public_and_private.csv')
guess = pd.read_csv('../testing/guess137.csv')

In [5]:
train_df = pd.read_parquet('../Data/training_feature_v1.parquet')

In [7]:
print(list(meta_data.keys()))

['prob_metrics', 'pred_metrics', 'thres_rec', 'f1_rec', 'prec_rec', 'recall_rec', 'wrong_cnt']


In [9]:
f_guess = guess.copy()

zero_list = [
    6722, 8831, 11999, 7555, 22570, 6898, 20029, 14120,
    14467, 4160, 12519, 8484, 9251, 18104, 18850, 15676, 16995,
    23508, 15749, 24661, 19364, 13181, 5418, 7613, 23976, 780,
    23557, 10288, 5375, 17790, 3479,2731,7752,6833,22045,23492,18305,14120,4328
]
f_guess['飆股'] = f_guess['飆股'].astype(float)
zero_np = np.array(zero_list) - 1

for i in zero_np:
    f_guess.iloc[i, 1] = 0.5

In [23]:
summary_df = pd.DataFrame({
    'ID': tmpl['ID'],
    'pred': loop_pred_100.mean(axis=1),
    'prob': loop_prob_100.mean(axis=1),
    'prob_std': loop_prob_100.std(axis=1) / loop_prob_100.mean(axis=1), 
} )[:25108]

In [24]:
summary_df.merge(f_guess, on='ID', how='outer').sort_values(by=['prob', 'prob_std'], ascending=False).reset_index(drop=True)

,ID,pred,prob,prob_std,飆股
0,PU-6548,1.0,1.000000e+00,7.297072e-08,1.0
1,PU-10413,1.0,1.000000e+00,1.297615e-07,1.0
2,PU-14894,1.0,1.000000e+00,1.132305e-07,1.0
3,PU-9297,1.0,1.000000e+00,1.186635e-07,1.0
4,PU-6741,1.0,1.000000e+00,1.682490e-07,1.0
...,...,...,...,...,...
25103,PU-21987,0.0,7.622114e-12,6.419390e+00,0.0
25104,PU-18133,0.0,7.567897e-12,6.264661e+00,0.0
25105,PU-15391,0.0,7.318213e-12,7.562390e+00,0.0
25106,PU-23470,0.0,6.898204e-12,7.387503e+00,0.0


In [30]:
summary_df.merge(f_guess, on='ID', how='outer').sort_values(by=['prob_std', 'pred'], ascending=[True, False]).reset_index(drop=True)

,ID,pred,prob,prob_std,飆股
0,PU-6548,1.0,1.000000e+00,7.297072e-08,1.0
1,PU-14894,1.0,1.000000e+00,1.132305e-07,1.0
2,PU-9297,1.0,1.000000e+00,1.186635e-07,1.0
3,PU-10413,1.0,1.000000e+00,1.297615e-07,1.0
4,PU-6741,1.0,1.000000e+00,1.682490e-07,1.0
...,...,...,...,...,...
25103,PU-7345,0.0,1.849204e-08,9.870454e+00,0.0
25104,PU-8116,0.0,4.373754e-09,9.872859e+00,0.0
25105,PU-15307,0.0,2.256899e-09,9.879965e+00,0.0
25106,PU-4845,0.0,5.106632e-07,9.906809e+00,0.0


In [73]:
pd.DataFrame({
    'ID': tmpl['ID'],
    '飆股': ((summary_df['pred']>0) & (summary_df['prob_std'] < 1.6)).astype(int),
}).sum()

ID    PU-1PU-2PU-3PU-4PU-5PU-6PU-7PU-8PU-9PU-10PU-11...
飆股                                                152.0
dtype: object

In [110]:
smt = pd.DataFrame({
    'ID': tmpl['ID'],
    '飆股': ((summary_df['pred']>0) & (summary_df['prob_std'] < 1.5)).astype(int),
})
print(smt.sum())
print(((smt['飆股'] == 1) & (f_guess['飆股'] == 1)).sum())
print(((smt['飆股'] == 1) & (f_guess['飆股'] == 0.5)).sum())

ID    PU-1PU-2PU-3PU-4PU-5PU-6PU-7PU-8PU-9PU-10PU-11...
飆股                                                147.0
dtype: object
131
12


In [42]:
smt.merge(f_guess, on='ID', how='outer').sort_values(by=['飆股_x'], ascending=False).reset_index(drop=True)

,ID,飆股_x,飆股_y
0,PU-12320,1.0,0.0
1,PU-16292,1.0,1.0
2,PU-15144,1.0,1.0
3,PU-7661,1.0,1.0
4,PU-24661,1.0,0.5
...,...,...,...
50211,PR-9995,NaN,NaN
50212,PR-9996,NaN,NaN
50213,PR-9997,NaN,NaN
50214,PR-9998,NaN,NaN


In [87]:
((smt['飆股'] == 1) & (f_guess['飆股'] == 1)).sum()

127

In [88]:
((smt['飆股'] == 1) & (f_guess['飆股'] == 0.5)).sum()

8

In [111]:
1e-2

0.01